## Previsão de Preços de Ações com Redes Neurais Recorrentes (LSTM)

#Breno Azevedo

In [24]:
!pip install keras

In [25]:
!pip install scikit-learn

In [26]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [27]:
!pip install yfinance


In [28]:
import yfinance as yf
import pandas as pd

# Baixar dados da ação PETR4 (PETR4.SA na B3)
dados = yf.download('PETR4.SA', start='2020-01-01', end='2024-12-31')

# Salvar tudo como base original (opcional)
dados.to_csv('PETR4_completa.csv')

# Separar 80% para treino, 20% para teste
percentual_treino = 0.8
divisor = int(len(dados) * percentual_treino)

base_treinamento = dados.iloc[:divisor]
base_teste = dados.iloc[divisor:]

# Salvar os arquivos
base_treinamento.to_csv('PETR4.csv')
base_teste.to_csv('PETR4_teste.csv')

print("Base de teste salva com sucesso com {} registros.".format(len(base_teste)))

[*********************100%***********************]  1 of 1 completed

Base de teste salva com sucesso com 249 registros.


In [29]:
dados = dados.dropna()

In [30]:
base_treinamento = dados.iloc[:, 1:2].values

In [31]:
base_treinamento

array([[12.51798058],
       [12.73816388],
       [12.61583813],
       ...,
       [36.        ],
       [36.        ],
       [36.36999893]])

In [32]:
normalizador = MinMaxScaler(feature_range=(0,1))
base_treinamento_normalizada = normalizador.fit_transform(base_treinamento)

In [33]:
base_treinamento_normalizada

array([[0.22057805],
       [0.22700951],
       [0.22343643],
       ...,
       [0.90647802],
       [0.90647802],
       [0.91728554]])

In [34]:
previsores = []
preco_real = []
for i in range(90, len(base_treinamento_normalizada)):
    previsores.append(base_treinamento_normalizada[i-90:i, 0])
    preco_real.append(base_treinamento_normalizada[i, 0])

In [35]:
previsores

[array([0.22057805, 0.22700951, 0.22343643, 0.22272187, 0.22141171,
        0.21962518, 0.21748133, 0.21736222, 0.21545659, 0.21140706,
        0.20866769, 0.21045424, 0.21343184, 0.21152618, 0.20997781,
        0.20938235, 0.20962061, 0.19604275, 0.20199798, 0.20128336,
        0.20080698, 0.19842491, 0.1971148 , 0.20080694, 0.20295088,
        0.20640476, 0.20449911, 0.20187884, 0.207834  , 0.21557569,
        0.21474195, 0.21176437, 0.20950144, 0.20950142, 0.21879148,
        0.22284093, 0.2084295 , 0.18115506, 0.17091223, 0.15935925,
        0.17353246, 0.18222691, 0.17579545, 0.17257969, 0.14232762,
        0.07205703, 0.07705942, 0.05895578, 0.02000923, 0.03835102,
        0.02286773, 0.01977105, 0.0010719 , 0.01060016, 0.01572157,
        0.        , 0.01619794, 0.03108577, 0.03584992, 0.0179845 ,
        0.01858001, 0.02810822, 0.02822733, 0.0520478 , 0.0497849 ,
        0.0466882 , 0.05824116, 0.06371986, 0.07753576, 0.05847938,
        0.06098049, 0.05109501, 0.054549  , 0.04

In [36]:
preco_real

[np.float64(0.06300523645692363),
 np.float64(0.07158064017819935),
 np.float64(0.07574927453948224),
 np.float64(0.08039424108205567),
 np.float64(0.08646849140773558),
 np.float64(0.09039885851540078),
 np.float64(0.08003693038895046),
 np.float64(0.08789773215718671),
 np.float64(0.09421019514867529),
 np.float64(0.09230448911464853),
 np.float64(0.09409103210581304),
 np.float64(0.09718774883859704),
 np.float64(0.09980796073104303),
 np.float64(0.10981258853004452),
 np.float64(0.11588682355831995),
 np.float64(0.1143384978289537),
 np.float64(0.12922631007750437),
 np.float64(0.12398580812263663),
 np.float64(0.11743512250306529),
 np.float64(0.1157677176379148),
 np.float64(0.10707322409109718),
 np.float64(0.10612040349554422),
 np.float64(0.11588680121324488),
 np.float64(0.11517222121185447),
 np.float64(0.11421939221114061),
 np.float64(0.11957899725309737),
 np.float64(0.11219461839507514),
 np.float64(0.1177924432810171),
 np.float64(0.11171820848198968),
 np.float64(0.110

In [37]:
previsores, preco_real = np.array(previsores), np.array(preco_real)

previsores = np.reshape(previsores,(previsores.shape[0], previsores.shape[1], 1))

In [38]:
previsores.shape[1]

90

In [39]:
#Criando a rede neuralo recorrente
regressor = Sequential()

regressor.add(LSTM(units = 100, return_sequences = True, input_shape = (previsores.shape[1], 1)))
regressor.add(Dropout(0.3))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.3))

regressor.add(Dense(units = 1, activation = 'linear'))

regressor.compile(optimizer = 'rmsprop', loss = 'mean_squared_error',
                  metrics = ['mean_absolute_error'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [40]:
regressor.fit(previsores, preco_real, epochs = 100, batch_size = 32)

Epoch 1/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 17s 185ms/step - loss: 0.1340 - mean_absolute_error: 0.2888
Epoch 2/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 7s 104ms/step - loss: 0.0907 - mean_absolute_error: 0.2511
Epoch 3/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 5s 99ms/step - loss: 0.0898 - mean_absolute_error: 0.2520
Epoch 4/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 5s 133ms/step - loss: 0.0840 - mean_absolute_error: 0.2455
Epoch 5/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 4s 101ms/step - loss: 0.0821 - mean_absolute_error: 0.2428
Epoch 6/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 5s 100ms/step - loss: 0.0841 - mean_absolute_error: 0.2471
Epoch 7/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 6s 117ms/step - loss: 0.0833 - mean_absolute_error: 0.2473
Epoch 8/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 4s 99ms/step - loss: 0.0872 - mean_absolute_error: 0.2506
Epoch 9/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 6s 132ms/step - loss: 0.0841 - mean_absolute_error: 0.2468
Epoch 10/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 4s 101ms/step - loss: 0.0826 - mean_absolute_error: 0.2430
Epoch 11/100
37/37 ━

In [41]:
base_teste = pd.read_csv('PETR4_teste.csv')
preco_real_teste = base_teste.iloc[:, 1:2].values

print(preco_real_teste)

[['PETR4.SA']
 [nan]
 ['32.72895431518555']
 ['32.805206298828125']
 ['32.55950927734375']
 ['32.27991485595703']
 ['31.983381271362305']
 ['32.25450134277344']
 ['32.3392219543457']
 ['32.68659210205078']
 ['32.27991485595703']
 ['32.09352493286133']
 ['31.96643829345703']
 ['31.796985626220703']
 ['31.94101905822754']
 ['32.3392219543457']
 ['32.09352493286133']
 ['33.279659271240234']
 ['33.85578155517578']
 ['34.37260437011719']
 ['34.160797119140625']
 ['34.53357696533203']
 ['35.219844818115234']
 ['34.762332916259766']
 ['34.91483688354492']
 ['35.23678970336914']
 ['35.75361251831055']
 ['35.465545654296875']
 ['34.9910888671875']
 ['34.72844696044922']
 ['35.83833694458008']
 ['36.16875457763672']
 ['36.346683502197266']
 ['35.96541976928711']
 ['36.01625442504883']
 ['35.74513626098633']
 ['35.49943161010742']
 ['36.16875457763672']
 ['36.117919921875']
 ['34.25398635864258']
 ['34.0082893371582']
 ['34.04218292236328']
 ['33.95745849609375']
 ['33.85578155517578']
 ['34.6013

In [43]:
base_completa = pd.concat((dados['Open'], base_teste['Open']), axis = 0)

In [44]:
print("Base treinamento: {}\nBase teste: {}\nBase completa: {}".format(
    len(base_treinamento), len(base_teste), len(base_completa))
)

Base treinamento: 1244
Base teste: 251
Base completa: 1495


In [45]:
inicio = len(base_completa) - len(base_teste) - 90
print(inicio)

1154


In [46]:
entradas = base_completa[inicio: ].values

In [47]:
entradas = entradas.reshape(-1, 1)

entradas = normalizador.transform(entradas)

ValueError: could not convert string to float: 'PETR4.SA'

In [ ]:
X_teste = []
final = 90 +len(base_teste)
for i in range(90, final):
    X_teste.append(entradas[i-90:i, 0])
X_teste = np.array(X_teste)
X_teste = np.reshape(X_teste, (X_teste.shape[0], X_teste.shape[1], 1))

In [ ]:
previsoes = regressor.predict(X_teste)
previsoes = normalizador.inverse_transform(previsoes)

In [ ]:
print("Média previsões: R${:-f}".format(previsoes.mean()))
print("Média real: R${:.2f}".format(preco_real_teste.mean()))

plt.plot(preco_real_teste, color = 'red', label = 'Preço real')
plt.plot(previsoes, color = 'blue', label = 'Previsões')
plt.title('Previsão preço das ações PETR4')
plt.xlabel('Tempo (dias)')
plt.ylabel('Valor da ação (R$)')
plt.legend()
plt.show()